<a href="https://colab.research.google.com/github/Fuenfgeld/TeamDataScDatenmanagementUndArchivierung/blob/main/ProjektCovid19Allergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Covid und Allergie
Die csv-Dateien sind im Ordner csv_files

Die db-Dateien sind im Ordner db_files

In [75]:
import pandas as pd
import numpy as np
import sqlite3 as sq
from sqlite3 import Error
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)
def create_connection_file():
    """ create a database connection to a database that resides
        in a file (db_files/cov_alle.db)
    """
    conn = None;
    try:
       #Establishing the connection
        conn = sq.connect('/content/drive/MyDrive/db_files/cov_alle.db')
        return conn
        print(sq.version)
    except Error as e:
        print("Error while connecting to sqlite", e)

# Creating a temporary empty database in the memory.
conn = create_connection_file()
# Creating a cursor object using the cursor() method
cur = conn.cursor()


Mounted at /content/drive/


Patienten

In [76]:
count_patients_cov = pd.read_sql_query("select count(*) patients_covid19 from patients_cov;", conn) #Anzahl an Patienten mit COVID-19
count_patients_cov

,patients_covid19
0,1050


In [77]:
count_patients_alle = pd.read_sql_query("select count(*) patients_allergy from patients_alle;", conn) #Anzahl an Patienten in Allergie
count_patients_alle


,patients_allergy
0,135


In [78]:
count_patients_gender_cov = pd.read_sql_query("select GENDER, count(id) patients_covid19 from patients_cov group by GENDER;", conn) #Patienten by Geschlecht in COVID-19
count_patients_gender_cov

,GENDER,patients_covid19
0,F,522
1,M,528


In [79]:
count_patients_gender_alle = pd.read_sql_query("select GENDER, count(id) patients_allergy from patients_alle group by GENDER;", conn) #Patienten by Geschlecht in COVID-19
count_patients_gender_alle

,GENDER,patients_allergy
0,F,57
1,M,78


In [80]:
# Patient und Conditions in conditions_cov

observations_patients_cov = pd.read_sql_query("select DESCRIPTION, count(distinct PATIENT) C_PATIENTS_covid19 FROM conditions_cov group by DESCRIPTION;", conn) #Anzahl an Patienten mit COVID-19
observations_patients_cov.head(4)

,DESCRIPTION,C_PATIENTS_covid19
0,Acquired coagulation disorder (disorder),11
1,Acute deep venous thrombosis (disorder),38
2,Acute pulmonary embolism (disorder),36
3,Acute respiratory distress syndrome (disorder),16


In [81]:
# Patient und Conditions in conditions_alle

observations_patients_alle = pd.read_sql_query("select DESCRIPTION, count(distinct PATIENT) C_PATIENTS_allergy FROM conditions_alle group by DESCRIPTION;", conn) 
observations_patients_alle.head(4)

,DESCRIPTION,C_PATIENTS_allergy
0,Acquired coagulation disorder (disorder),1
1,Acute allergic reaction,1
2,Acute bacterial sinusitis (disorder),9
3,Acute bronchitis (disorder),52


In [82]:
# Patient und Description in encounter_alle

encounter_description_patients_alle = pd.read_sql_query("select DESCRIPTION , count(distinct PATIENT) C_PATIENTS_allergy FROM encounters_alle ea  where DESCRIPTION not like '' group by DESCRIPTION ;", conn) 
encounter_description_patients_alle

,DESCRIPTION,C_PATIENTS_allergy
0,Admission to intensive care unit (procedure),8
1,Admission to surgical department,5
2,Allergic disorder follow-up assessment,2
3,Allergic disorder initial assessment,2
4,Asthma follow-up,3
5,Cardiac Arrest,3
6,Consultation for treatment,34
7,Death Certification,35
8,Emergency Encounter,9
9,Emergency Room Admission,15


In [83]:
# Patient und Description in encounter_cov

encounter_description_patients_cov = pd.read_sql_query("select DESCRIPTION , count(distinct PATIENT) C_PATIENTS_covid19 FROM encounters_cov co  where DESCRIPTION not like '' group by DESCRIPTION ;", conn) 
encounter_description_patients_cov

,DESCRIPTION,C_PATIENTS_covid19
0,Admission to intensive care unit (procedure),26
1,Cardiac Arrest,47
2,Death Certification,50
3,Encounter for symptom (procedure),905
4,General examination of patient (procedure),840
5,Hospital admission for observation (procedure),8
6,Hospital admission for isolation (procedure),129
7,Myocardial Infarction,20
8,Stroke,54
9,Well child visit (procedure),356


In [84]:
# Patient und Reasondescription in encounter_alle

encounter_reasondescription_patients_alle = pd.read_sql_query("select REASONDESCRIPTION , count(distinct PATIENT) C_PATIENTS_allergy FROM encounters_alle ea  where REASONDESCRIPTION not like '' group by REASONDESCRIPTION ;", conn) 
encounter_reasondescription_patients_alle

,REASONDESCRIPTION,C_PATIENTS_allergy
0,Acute bacterial sinusitis (disorder),20
1,Acute bronchitis (disorder),54
2,Acute viral pharyngitis (disorder),52
3,Alzheimer's disease (disorder),10
4,Anemia (disorder),14
...,...,...
60,Stroke,4
61,Suspected lung cancer (situation),2
62,Third degree burn,1
63,Tubal pregnancy,3


In [85]:
# Patient und Reasondescription in encounter_cov

encounter_reasondescription_patients_cov = pd.read_sql_query("select REASONDESCRIPTION , count(distinct PATIENT) C_PATIENTS_covid19 FROM encounters_cov co  where REASONDESCRIPTION not like '' group by REASONDESCRIPTION ;", conn) 
encounter_reasondescription_patients_cov

,REASONDESCRIPTION,C_PATIENTS_covid19
0,COVID-19,129
1,Cardiac Arrest,4
2,Myocardial Infarction,17
3,Stroke,5


In [86]:
# Age analysis (Covid)

ageCov = pd.read_sql_query("""select 
    PA.ID Patient_allergy, 
    MAX(strftime('%Y', START)) - strftime('%Y', BIRTHDATE) AGE_END 
  from patients_cov PA 
  JOIN encounters_cov EA 
    ON PA.Id = EA.PATIENT 
  GROUP BY PA.ID""", conn)

print('Age in covid')
print('count ' + str(ageCov['AGE_END'].count()))
print('max ' + str(ageCov['AGE_END'].max()))
print('min ' + str(ageCov['AGE_END'].min()))
print('mean ' + str(ageCov['AGE_END'].mean()))
print('median ' + str(ageCov['AGE_END'].median()))
print('mode')
print(str(ageCov['AGE_END'].mode()))


Age in covid
count 1050
max 110
min 1
mean 41.142857142857146
median 39.0
mode
0    22
1    31
dtype: int64


In [87]:
# Age analysis (Allergy)

ageAlle = pd.read_sql_query("""select 
    PA.ID Patient_allergy, 
    MAX(strftime('%Y', START)) - strftime('%Y', BIRTHDATE) AGE_END 
  from patients_alle PA 
  JOIN encounters_alle EA 
    ON PA.Id = EA.PATIENT 
  GROUP BY PA.ID""", conn)
print('Age in allergy')
print('count ' + str(ageAlle['AGE_END'].count()))
print('max ' + str(ageAlle['AGE_END'].max()))
print('min ' + str(ageAlle['AGE_END'].min()))
print('mean ' + str(ageAlle['AGE_END'].mean()))
print('median ' + str(ageAlle['AGE_END'].median()))
print('mode')
print(str(ageAlle['AGE_END'].mode()))

Age in allergy
count 135
max 104
min 0
mean 47.8962962962963
median 49.0
mode
0    18
1    61
2    69
dtype: int64


In [88]:
conn.close()